In [4]:
!pip -qq install torchmetrics 

In [5]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision.models as models
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt
import time
from torchvision.datasets import ImageFolder
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import zipfile

In [6]:
path_to_zip_file = '/content/drive/MyDrive/DL_study_179/data/animal-10/archive.zip'
directory_to_extract_to = '.'

if not os.path.isdir('/content/raw-img'):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

In [7]:
translate = {
    "cane": "dog", 
    "cavallo": "horse", 
    "elefante": "elephant", 
    "farfalla": "butterfly", 
    "gallina": "chicken", 
    "gatto": "cat", 
    "mucca": "cow", 
    "pecora": "sheep", 
    "ragno": "spider",
    "scoiattolo": "squirrel", 
    "dog": "cane", 
    "horse": "cavallo", 
    "elephant" : "elefante", 
    "butterfly": "farfalla", 
    "chicken": "gallina", 
    "cat": "gatto", 
    "cow": "mucca", 
    "sheep": "pecora",
    "spider": "ragno", 
    "squirrel": "scoiattolo"
    }

In [8]:
CLASS_NUMBER = {
    "butterfly": 0,
    "cat": 1,
    "chicken": 2,
    "cow": 3,
    "dog": 4,
    "elephant": 5,
    "horse": 6,
    "sheep": 7,
    "spider": 8,
    "squirrel": 9
}

In [9]:
import os
import random
import cv2
import numpy as np


def image_resize(img):
    # 세로 > 가로
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]), 256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = cv2.resize(img, dsize=tile_size)
    
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

def save_img(save_path, folder_name, image_list):
    new_name = translate[folder_name]
    folder_path = os.path.join(save_path, new_name)
    
    if not os.path.isdir(folder_path):
        os.mkdir(folder_path)
    
    for i, image in enumerate(image_list):
        img = cv2.imread(image)
        img = image_resize(img)
        image_path = os.path.join(folder_path, new_name + "_" + str(i) + ".jpg")
        cv2.imwrite(image_path, img)


if __name__ == "__main__":
    random.seed(100)
    
    BASE_PATH = "."
    BASE_PATH = os.path.abspath(BASE_PATH)  
    source_path = os.path.join(BASE_PATH, "raw-img")
    assert os.path.isdir(source_path)
    
    train_path = os.path.join(BASE_PATH, "train_img")
    test_path = os.path.join(BASE_PATH, "test_img")
    
    if not os.path.isdir(train_path):
        os.mkdir(train_path)
    if not os.path.isdir(test_path):
        os.mkdir(test_path)

    folder_list = os.listdir(source_path)

    if '.DS_Store' in folder_list:
        folder_list.remove('.DS_Store')


    img_set = {}
    for folder in folder_list:
        folder_path = os.path.join(source_path, folder)
        image_list = os.listdir(folder_path)
        
        image_path_list = []
        for image in image_list:
            image_path_list.append(os.path.join(folder_path, image))
        img_set[folder] = image_path_list


    for folder in img_set:
        random.shuffle(img_set[folder])
        train_length = int(len(img_set[folder]) * 0.8)
        
        train_list = img_set[folder][:train_length]
        test_list = img_set[folder][train_length:]

        save_img(train_path, folder, train_list)
        save_img(test_path, folder, test_list)

In [10]:
class ImageTransform():    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
        
    def __call__(self, img, phase):
        return self.data_transform[phase](img)

In [17]:
size = 256
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
batch_size = 32
num_epochs = 5

In [38]:
train_transforms = transforms.Compose([
                           transforms.Resize((256, 256)),
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.ToTensor(),
                        #    transforms.Normalize(mean=mean,std=std)
                           ])

test_transforms = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(256),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])

In [39]:
train_path = '/content/train_img'
test_path = '/content/test_img'

train_dataset = torchvision.datasets.ImageFolder(
    train_path,
    transform=train_transforms
)

test_dataset = torchvision.datasets.ImageFolder(
    test_path,
    transform=test_transforms
)

In [20]:
def save_model(model, path, kfold, epoch):
    # PATH = '/content/drive/MyDrive/DL_study_179/weight/week2/'
    torch.save(model, '{}{}_{}_model.pt'.format(path, kfold, epoch))  # 전체 모델 저장
    # torch.save(model.state_dict(), '{}{}_{}_model_state_dict.pt'.format(path, kfold, epoch))  # 모델 객체의 state_dict 저장
    # torch.save({
    #     'model': model.state_dict(),
    #     'optimizer': optimizer.state_dict()
    # }, '{}{}_{}_all.tar'.format(path, kfold, epoch))  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [21]:
def train_model(model, dataloaders_dict, criterion, optimizer, kfold, num_epochs, save_path):
    model.to(device)

    history = {'train_loss': [], 'val_loss': [],'train_acc': [],'val_acc': []}

    torch.backends.cudnn.benchmark = True

    for epoch in range(num_epochs):
        # print('-' * 20)
        print('Epoch {:02d}/{} | '.format(epoch+1, num_epochs), end='')
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            epoch_loss = 0.0
            epoch_corrects = 0

            # if (epoch == 0) and (phase == 'train'):
            #     continue

            for inputs, labels in dataloaders_dict[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    

            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            
            history[phase+'_loss'].append(epoch_loss)
            history[phase+'_acc'].append(epoch_acc.cpu())
            print('{} | Loss: {:.4f} Acc: {:.4f} | '.format(phase, epoch_loss, epoch_acc), end='')
        print()

        # save_model(model, save_path, kfold, epoch)
    return history

In [22]:
kf = StratifiedKFold(n_splits=5)
foldperf={}
save_path = '/content/drive/MyDrive/DL_study_179/weight/week2/'
for i, (train_index, val_index) in enumerate(kf.split(train_dataset, train_dataset.targets)):

    pretrained_model=models.resnet18(pretrained=True)
    pretrained_model.fc = nn.Linear(512, 10)

    # optimizer = optim.Adam(pretrained_model.parameters(), lr=0.001)
    optimizer = optim.SGD(pretrained_model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss().to(device)

    # print("train_index : {}, val_index : {}".format(len(train_index), len(val_index)))
    train = torch.utils.data.Subset(train_dataset, train_index)
    valid = torch.utils.data.Subset(train_dataset, val_index)


    trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid, batch_size=batch_size)
    
    
    dataloaders_dict = {"train" : trainloader, "val": validloader}
    
    print('-' * 20)
    print('K-fold: {}'.format(i+1))
    print('-' * 20)
    history = train_model(pretrained_model, dataloaders_dict, criterion, optimizer, i+1, num_epochs, save_path)

    foldperf['fold{}'.format(i+1)] = history


--------------------
K-fold: 1
--------------------
Epoch 01/5 | train | Loss: 0.3931 Acc: 0.8904 | val | Loss: 0.1509 Acc: 0.9568 | 
Epoch 02/5 | train | Loss: 0.1462 Acc: 0.9579 | val | Loss: 0.1320 Acc: 0.9601 | 
Epoch 03/5 | train | Loss: 0.1073 Acc: 0.9685 | val | Loss: 0.1256 Acc: 0.9632 | 
Epoch 04/5 | train | Loss: 0.0778 Acc: 0.9778 | val | Loss: 0.1267 Acc: 0.9628 | 
Epoch 05/5 | train | Loss: 0.0583 Acc: 0.9856 | val | Loss: 0.1194 Acc: 0.9661 | 
--------------------
K-fold: 2
--------------------
Epoch 01/5 | train | Loss: 0.3946 Acc: 0.8916 | val | Loss: 0.1548 Acc: 0.9553 | 
Epoch 02/5 | train | Loss: 0.1443 Acc: 0.9592 | val | Loss: 0.1342 Acc: 0.9592 | 
Epoch 03/5 | train | Loss: 0.1072 Acc: 0.9704 | val | Loss: 0.1190 Acc: 0.9599 | 
Epoch 04/5 | train | Loss: 0.0810 Acc: 0.9778 | val | Loss: 0.1083 Acc: 0.9673 | 
Epoch 05/5 | train | Loss: 0.0610 Acc: 0.9834 | val | Loss: 0.1102 Acc: 0.9673 | 
--------------------
K-fold: 3
--------------------
Epoch 01/5 | train | Los

In [23]:
# pretrained_model=models.resnet18(pretrained=False)
# pretrained_model.fc = nn.Linear(512, 10)
# # optimizer = optim.Adam(pretrained_model.parameters(), lr=0.001)
# optimizer = optim.SGD(pretrained_model.parameters(), lr=0.001, momentum=0.9)
# criterion = nn.CrossEntropyLoss().to(device)
# # print("train_index : {}, val_index : {}".format(len(train_index), len(val_index)))


# pt_path = '/content/drive/MyDrive/DL_study_179/weight/week2/3_49_model.pt'
# loaded_model = torch.load(pt_path)

# loaded_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [47]:
category_list = os.listdir('/content/train_img')

test_cate_dirs_dict = {}
for category in category_list:
    test_cate_dirs_dict[category] = '/content/test_img/' + category


test_cate_images_filepaths = {}
for cate, cate_dir in test_cate_dirs_dict.items():
    test_cate_images_filepaths[cate] = sorted([os.path.join(cate_dir, f) for f in os.listdir(cate_dir)])

test_images_filepaths = []

for path, image_list in test_cate_images_filepaths.items():
    test_images_filepaths += image_list

test_correct_images_filepaths = [i for i in test_images_filepaths if cv2.imread(i) is not None]


random.seed(42)
random.shuffle(test_correct_images_filepaths)

test_images_filepaths = test_correct_images_filepaths[:]


In [50]:
class Animal10Dataset(Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)

        label = img_path.split('/')[-1].split('_')[0]
        label = CLASS_NUMBER[label]
        return img_transformed, label

In [51]:
test_dataset = Animal10Dataset(test_images_filepaths, transform=ImageTransform(size, mean, std), phase='val')
test_iterator = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [52]:
all_preds = torch.Tensor([]).to(device)
all_labels = torch.Tensor([]).to(device)
for images, labels in test_iterator:
    
    images, labels = images.to(device), labels.to(device)
    preds=pretrained_model(images).argmax(dim=1).to(device)
    all_preds = torch.cat((all_preds, preds), dim=0)
    all_labels = torch.cat((all_labels, labels), dim=0)

In [53]:
from torchmetrics import ConfusionMatrix
all_preds = all_preds.clone().detach().type(torch.IntTensor).to(device)
all_labels = all_labels.clone().detach().type(torch.IntTensor).to(device)

In [54]:
confmat = ConfusionMatrix(num_classes=10).to(device)
confusion_matrix = confmat(all_preds, all_labels).cpu()

In [55]:
confusion_matrix

tensor([[ 53,   6,   0,   0, 311,   0,   0,   0,  53,   0],
        [ 48,  17,   0,   0, 228,   0,   0,   0,  41,   0],
        [ 84,  17,   0,   0, 396,   0,   0,   0, 123,   0],
        [ 31,  11,   0,   0, 292,   0,   0,   0,  40,   0],
        [124,  53,   0,   0, 653,   0,   0,   0, 143,   0],
        [ 36,  13,   0,   0, 210,   0,   0,   0,  31,   0],
        [ 68,  23,   0,   0, 387,   0,   0,   0,  47,   0],
        [ 42,  20,   0,   0, 238,   0,   0,   0,  64,   0],
        [262,  31,   0,   0, 472,   0,   0,   0, 200,   0],
        [ 44,  13,   0,   0, 235,   0,   0,   0,  81,   0]])

In [45]:
from torchmetrics import F1Score
f1 = F1Score(num_classes=10).to(device)
f1_score = f1(all_preds, all_labels).cpu()

In [46]:
f1_score

tensor(0.1761)

```
tensor(0.9599)
```